In [ ]:
import pandas as pd
import altair as alt
import seaborn as sns
from matplotlib import pyplot as plt
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
import numpy as np
import math
from sklearn.preprocessing import StandardScaler



housing = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv')
holdout = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing_holdout_test.csv')

In [ ]:
#TODO drop row with 33 bedrooms?

In [ ]:
# Average Household Income

# housing['zipcode'].unique()

zip_income_data = {
    98001: 127394, 98002: 81661, 98003: 86274, 98004: 241568, 98005: 212891, 98006: 239678, 98007: 153950,
    98008: 213509, 98010: 141579, 98011: 145944, 98014: 178288, 98019: 172742, 98022: 141658, 98023: 116874,
    98024: 168804, 98027: 187552, 98028: 168571, 98029: 209027, 98030: 101317, 98031: 124247, 98032: 103052,
    98033: 163786, 98034: 154944, 98038: 155640, 98039: 439677, 98040: 290655, 98042: 149928, 98045: 189177,
    98052: 201422, 98053: 249748, 98055: 109741, 98056: 136107, 98058: 129245, 98059: 180460, 98065: 210831,
    98070: 145634, 98072: 201007, 98074: 261213, 98075: 292189, 98077: 235594, 98092: 136596, 98102: 157582,
    98103: 167932, 98105: 155790, 98106: 122095, 98107: 167124, 98108: 129864, 98109: 166998, 98112: 256059,
    98115: 187326, 98116: 170002, 98117: 195884, 98118: 144942, 98119: 170519, 98122: 147095, 98125: 128321,
    98126: 141511, 98133: 114119, 98136: 175739, 98144: 157648, 98146: 124943, 98148: 102902, 98155: 151036,
    98166: 144311, 98168: 97792, 98177: 195884, 98178: 133779, 98188: 94339, 98198: 103364, 98199: 205611
}

housing['Average_Household_Income'] = housing['zipcode'].map(zip_income_data)
holdout['Average_Household_Income'] = holdout['zipcode'].map(zip_income_data)

housing.head()

# source
# https://www.incomebyzipcode.com/washington/98119#:~:text=98119%20Average%20Household%20Income,and%20Seattle%20city%20(%24161%2C694).

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,zipcode,lat,long,sqft_living15,sqft_lot15,price,Average_Household_Income,year,month,day
0,1565930130,20141104T000000,4,3.25,3760,4675,2.0,0,0,3,...,98038,47.3862,-122.048,3280,4033,429900.0,155640,2014,11,4
1,3279000420,20150115T000000,3,1.75,1460,7800,1.0,0,0,2,...,98023,47.3035,-122.382,1310,7865,233000.0,116874,2015,1,15
2,194000575,20141014T000000,4,1.00,1340,5800,1.5,0,2,3,...,98116,47.5658,-122.389,1900,5800,455000.0,170002,2014,10,14
3,2115510160,20141208T000000,3,1.75,1440,8050,1.0,0,0,3,...,98023,47.3187,-122.390,1790,7488,258950.0,116874,2014,12,8
4,7522500005,20140815T000000,2,1.50,1780,4750,1.0,0,0,4,...,98117,47.6859,-122.395,1690,5962,555000.0,195884,2014,8,15


In [ ]:
def find_zip(row):
    zip = row["zipcode"]
    id = row["id"]
    new_housing = housing[(housing["zipcode"] == zip) & (housing["id"] != id)]
    return tuple(new_housing["price"])


housing["prices_in_zip"] = housing.apply(find_zip, axis=1)
holdout["prices_in_zip"] = holdout.apply(find_zip, axis=1)

housing["prices_in_zip"].head()

0    (359900.0, 285000.0, 499950.0, 550000.0, 34000...
1    (258950.0, 265000.0, 263000.0, 221000.0, 30745...
2    (334850.0, 729500.0, 479200.0, 388000.0, 63995...
3    (233000.0, 265000.0, 263000.0, 221000.0, 30745...
4    (552500.0, 1280000.0, 555000.0, 657500.0, 9300...
Name: prices_in_zip, dtype: object

In [ ]:
# example of what we can do
# def get_discount_percentage(Average_Household_Income):
#     if Average_Household_Income <= 100000:
#         return 0.15  # 15% discount
#     elif 100001 <= Average_Household_Income <= 130000:
#         return 0.10  # 10% discount
#     elif 130001 <= Average_Household_Income <= 160000:
#         return 0.05  # 5% discount
#     else:
#         return 0.00  # No discount

# housing['discount_percentage'] = housing['Average_Household_Income'].apply(get_discount_percentage)
# housing['adjusted_price'] = housing['price'] * (1 - housing['discount_percentage'])
# housing.head()

# we will not add this due to ethical reasons

In [ ]:
housing['date']

housing['year'] = housing['date'].str[:4]
housing['month'] = housing['date'].str[4:6]
housing['day'] = housing['date'].str[6:8]


holdout['date']

holdout['year'] = holdout['date'].str[:4]
holdout['month'] = holdout['date'].str[4:6]
holdout['day'] = holdout['date'].str[6:8]


In [ ]:
housing[['year','month','day']] = housing[['year','month','day']].astype(int)
holdout[['year','month','day']] = holdout[['year','month','day']].astype(int)

In [ ]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        20000 non-null  int64  
 1   date                      20000 non-null  object 
 2   bedrooms                  20000 non-null  int64  
 3   bathrooms                 20000 non-null  float64
 4   sqft_living               20000 non-null  int64  
 5   sqft_lot                  20000 non-null  int64  
 6   floors                    20000 non-null  float64
 7   waterfront                20000 non-null  int64  
 8   view                      20000 non-null  int64  
 9   condition                 20000 non-null  int64  
 10  grade                     20000 non-null  int64  
 11  sqft_above                20000 non-null  int64  
 12  sqft_basement             20000 non-null  int64  
 13  yr_built                  20000 non-null  int64  
 14  yr_ren

In [ ]:
# Shortened List of Features After testinghousing
X = pd.get_dummies(housing[['bathrooms','sqft_living','waterfront','view','condition','grade','sqft_above',
                            'yr_built','yr_renovated','zipcode','lat','long','sqft_living15','month','sqft_lot15',
                            'Average_Household_Income']],drop_first=True)

X_holdout = pd.get_dummies(holdout[['bathrooms','sqft_living','waterfront','view','condition','grade','sqft_above',
                            'yr_built','yr_renovated','zipcode','lat','long','sqft_living15','month','sqft_lot15',
                            'Average_Household_Income']], drop_first=True)



y = housing['price']


In [ ]:
#We need validation set
X_train, X_other, y_train, y_other = train_test_split(X, y, test_size=0.30, random_state = 42)
X_test, X_val, y_test, y_val = train_test_split(X_other, y_other, test_size=0.50, random_state = 42)

In [ ]:
#use hyperparameters
    #'max_depth':                [5], #5
    #'min_child_weight':         [5], #5
    #'subsample':                [1], #1
    #'colsample_bytree':          [1], #1
    #'eta':                       [.3],  #0.3 #alias "learning_rate"
    #'early_stopping_rounds': [20]
    # grow_policy: depthwise
    # tree_method: exact
model = XGBRegressor(max_depth=5, min_child_weight = 5, subsample = 1, colsample_bytree=1, eta=0.3 , early_stopping_rounds=20)#grow_policy="depthwise", tree_method="exact") #'eta' is the same as 'learning_rate'

In [ ]:
#fit using the validation set so we can tune the 'early_stopping_rounds' hyperparameter
model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)])

model.score(X_test,y_test)

[0]	validation_0-rmse:280339.38502	validation_1-rmse:319136.53474
[1]	validation_0-rmse:228559.25766	validation_1-rmse:273310.88425
[2]	validation_0-rmse:191645.81615	validation_1-rmse:239281.07010
[3]	validation_0-rmse:167673.54458	validation_1-rmse:216271.51720
[4]	validation_0-rmse:149570.38898	validation_1-rmse:198093.46439
[5]	validation_0-rmse:137573.21850	validation_1-rmse:188318.47662
[6]	validation_0-rmse:128194.40323	validation_1-rmse:179059.70137
[7]	validation_0-rmse:122641.82122	validation_1-rmse:175204.48466
[8]	validation_0-rmse:117697.67260	validation_1-rmse:171486.82241
[9]	validation_0-rmse:114422.83965	validation_1-rmse:166034.80067
[10]	validation_0-rmse:111736.23425	validation_1-rmse:163942.40801
[11]	validation_0-rmse:109485.64383	validation_1-rmse:161028.22178
[12]	validation_0-rmse:107567.67622	validation_1-rmse:159024.97762
[13]	validation_0-rmse:106249.79532	validation_1-rmse:157498.82751
[14]	validation_0-rmse:105043.45480	validation_1-rmse:155826.17667
[15]	

0.8810553778079505

In [ ]:
test_predictions = model.predict(X_test)
test_predictions

array([274824.06, 419982.75, 542127.4 , ..., 934899.7 , 258082.86,
       416119.9 ], dtype=float32)

In [ ]:
#root mean squared error
result = mean_squared_error(y_test, test_predictions, squared=False)
result

134091.80722017068

In [ ]:
mse = mean_squared_error(y_test, test_predictions, squared=True)
mse

17980612763.57142

In [ ]:
r2 = r2_score(y_test,test_predictions)
r2

0.8810553778079505

In [ ]:
holdout_predictions = model.predict(X_holdout)
holdout_predictions

array([ 362056.97,  932247.3 , 1180889.8 , ...,  253125.75, 1372965.4 ,
        310535.62], dtype=float32)

In [ ]:
holdout_data = pd.DataFrame({"price": holdout_predictions})

In [ ]:
from google.colab import files

filename = r'team5-module3-predictions.csv'

try:
    holdout_data.to_csv(filename, index=False)
    print("CSV file saved successfully.")
except Exception as e:
    print("An error occurred while saving the CSV file:", e)

files.download(filename)

CSV file saved successfully.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor

# Assuming X and y are already defined

# Create and fit a GradientBoostingRegressor
model = GradientBoostingRegressor()
model.fit(X, y)

# Get feature importances
feature_importances = model.feature_importances_

# Create a DataFrame with feature names and their importances
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

# Sort the DataFrame by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Select the top 15 features
top_15_features = feature_importance_df.head(11)

# Plot top 15 feature importances
plt.figure(figsize=(12, 6))
plt.barh(top_15_features['Feature'], top_15_features['Importance'])
plt.xlabel('Importance')
plt.title('Top Feature Importances in Gradient Boosting Regressor')
plt.gca().invert_yaxis()  # Invert y-axis to show the most important feature at the top
plt.show()